# Aula 2 - Reinforcement Learning

## Tutorial: Q Learning no ambiente FrozenLake

### Prof. Dr. Ahirton Lopes (profahirton.lopes@fiap.com.br)

# Q* Learning com FrozenLake 4x4

Neste Notebook, implementaremos um agente <b>que reproduz o desafio FrozenLake.</b>

![texto alternativo](https://www.gymlibrary.dev/_images/frozen_lake.gif)

O objetivo deste jogo é <b>passar do estado inicial (S) para o estado objetivo (G)</b> andando apenas sobre peças congeladas (F) e evitando buracos (H). No entanto, o gelo é escorregadio, **então você nem sempre se moverá na direção pretendida (ambiente estocástico)**

## Pré-requisitos 🏗️

Antes de mergulhar no notebook **você precisa entender**:
- Os fundamentos da aprendizagem por reforço (MC, TD, hipótese de recompensas...)
- Q-aprendizagem

## Etapa 1: Instalando as dependências no Google Colab

In [1]:
!pip install gymnasium
!pip install numpy

## Etapa 0: Importando as dependências 📚

Usamos 3 bibliotecas:

- `Numpy` para nosso Qtable
- `Gymnasium` para nosso ambiente FrozenLake
- `Random` para gerar números aleatórios

In [2]:
import numpy as np
import gymnasium as gym
import random

## Passo 1: Criando o ambiente 🎮

- Aqui criaremos o ambiente FrozenLake 8x8.
- Gymnasium é uma biblioteca <b> composta por diversos ambientes que podemos usar para treinar nossos agentes.</b>
- No nosso caso optamos por usar Frozen Lake.

In [3]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=True, render_mode="human")

## Etapa 2: Criando a tabela Q e inicializando-a 🗄️

- Agora, vamos criar nossa Q-table, para saber de quantas linhas (estados) e colunas (ações) precisamos, precisamos calcular o action_size e o state_size
- Gymnasium nos fornece uma maneira de fazer isso: `env.action_space.n` e `env.observation_space.n`

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [5]:
action_size

np.int64(4)

In [6]:
state_size

np.int64(16)

In [7]:
# Create our Q table with state_size rows and action_size columns (16x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Etapa 3: Criando os hiperparâmetros ⚙️

- Aqui, especificaremos os hiperparâmetros

In [8]:
total_episodes = 500        # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob


![texto alternativo](https://miro.medium.com/v2/resize:fit:1400/1*tyIE_430xbBRzrrjUdYLQw.png)


## Etapa 4: O algoritmo de aprendizagem Q 🧠

- Agora implementamos o algoritmo de aprendizagem Q:


In [9]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state, _ = env.reset()
    state = int(state)
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)


        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward

        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)
    print(episode)


print ("Pontuação no tempo: " +  str(sum(rewards)/total_episodes))
print(qtable)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Etapa 5: Usando nossa tabela Q para jogar FrozenLake! 👾

- Após 10.000 episódios, nossa tabela Q pode ser usada como uma "folha de dicas" para jogar FrozenLake"
- Ao executar este celular você poderá ver nosso agente jogando FrozenLake.

In [11]:
for episode in range(5):
    state, info = env.reset()
    state = int(state)
    step = 0
    done = False
    max_steps = 100
    print("****************************************************")
    print("EPISODIO ", episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, terminated, truncated, _ = env.step(action)

        done = terminated or truncated

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("Chegamos no Objetivo 🏆")
            else:
                print("Caímos em um Obstáculo ☠️")

            # We print the number of step it took.
            print("Número de passos", step)

            break
        state = int(new_state)
env.close()

error: display Surface quit